# NAME : SUBASH R
# REG NO : 212223230218
# WORKSHOP : Object detection using web camera



In [28]:
import cv2
import numpy as np

# Load YOLO model
net = cv2.dnn.readNet(
    r"C:\Users\admin\OneDrive\Documents\yolov4.weights",
    r"C:\Users\admin\OneDrive\Documents\yolov4.cfg"
)

# Load class names
with open(r"C:\Users\admin\Documents\coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Open webcam
cap = cv2.VideoCapture(0)

layer_names = net.getLayerNames()
outs = net.getUnconnectedOutLayers()
output_layers = [layer_names[i - 1] for i in outs]

colors = np.random.uniform(0, 255, size=(len(classes), 3))

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    height, width = frame.shape[:2]

    # YOLO preprocessing
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outputs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:
                cx = int(detection[0] * width)
                cy = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(cx - w / 2)
                y = int(cy - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Non-Max Suppression
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            color = colors[class_ids[i]]
            label = f"{classes[class_ids[i]]} {confidences[i]:.2f}"

            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Show frame in OpenCV window
    cv2.imshow("YOLOv4 Detection", frame)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('c'):
        cv2.imwrite("captured_frame.jpg", frame)
        print("Image captured!")

    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Image captured!
